In [1]:
import meshcat
from utils import *
from interface import ExoIMUs
import time 
import collections


# from estimation_problem import solve_estimation_problem
from utils import ArmMeasurementCurrent
from arm_model import create_arm
from pinocchio.visualize import MeshcatVisualizer as Visualizer


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
imu = ExoIMUs("/dev/ttyACM0")

In [4]:
rmodel, rdata, gmodel, cmodel = create_arm()
viz = Visualizer(rmodel, gmodel, cmodel)
viz.initViewer(open=False)
viz.loadViewerModel()
viz.initializeFrames()
viz.display_frames = True

add_frame("hand", viz.viewer)
add_frame("shoulder", viz.viewer)

viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7012/static/


In [13]:
names = ['hand', 'base', 'shoulder']
data_offset = []
offset = [pin.utils.rpyToMatrix(np.pi/2.0, 0, 0), 0]
counter = 0
T = 10
estimate_x0 = np.zeros(rmodel.nq + rmodel.nv)

measurement = collections.deque(maxlen=T)

while True:
    data = imu.read()
    # print(data["base_acc"], data["shoulder_acc"], data["hand_acc"])
    base = Rotation.from_quat(data['q_base']).as_matrix()
    shoulder = Rotation.from_quat(data['q_shoulder']).as_matrix()
    hand = Rotation.from_quat(data['q_hand']).as_matrix()
    if counter < 100:
        data_offset.append([base.T @ shoulder, base.T @ hand])
    elif counter == 100:
        offset[0] = offset[0] @ data_offset[50][0].T
        offset[1] = data_offset[50][1].T
    else:
        update_frame('hand', viz.viewer, offset[1] @ base.T @ hand, [0.5, 0, 0])
        update_frame("shoulder", viz.viewer,   offset[0] @ base.T @ shoulder)
        measurement.append(ArmMeasurementCurrent( offset[0] @ base.T @ shoulder, offset[1] @ base.T @ hand))
        if counter > T + 100:
            estimate = solve_estimation_problem(measurement, T, rmodel, estimate_x0)
            viz.display(estimate.xs[-1][:rmodel.nq])
            estimate_x0 = estimate.xs[-1]
 
    counter += 1
    time.sleep(0.01)

KeyboardInterrupt: 

In [12]:
import crocoddyl
# from mim_solvers import SolverSQP
import numpy as np

def solve_estimation_problem(measurements, T, rmodel, x0):
    rdata = rmodel.createData()
    nq = rmodel.nq; nv = rmodel.nv; nu = nq; nx = nq+nv

    
    dt = 1e-2
    
    # # # # # # # # # # # # # # #
    ###  SETUP CROCODDYL OCP  ###
    # # # # # # # # # # # # # # #
    
    # State and actuation model
    state = crocoddyl.StateMultibody(rmodel)
    actuation = crocoddyl.ActuationModelFull(state)
    
    # Create cost terms 
    # Control regularization cost
    uResidual = crocoddyl.ResidualModelControlGrav(state)
    uRegCost = crocoddyl.CostModelResidual(state, uResidual)
    # State regularization cost
    xResidual = crocoddyl.ResidualModelState(state, x0)
    xRegCost = crocoddyl.CostModelResidual(state, xResidual)

    runningModel = []
    for i in range(T):
        # endeff frame orientation cost
        endeff_frame_id = rmodel.getFrameId("Hand")
        frameOrientationResidual = crocoddyl.ResidualModelFrameRotation(state, endeff_frame_id, measurements[i].palm_orientation)
        frameOrientationCost = crocoddyl.CostModelResidual(state, frameOrientationResidual)

        imu_arm_id = rmodel.getFrameId("imu_arm")
        imuArmOrientationResidual = crocoddyl.ResidualModelFrameRotation(state, imu_arm_id, measurements[i].arm_orientation)
        imuArmOrientationCost = crocoddyl.CostModelResidual(state, imuArmOrientationResidual)
        
        # Running and terminal cost models
        runningCostModel = crocoddyl.CostModelSum(state)
        terminalCostModel = crocoddyl.CostModelSum(state)
        
        # Add costs
        runningCostModel.addCost("stateReg", xRegCost, 5e-3)
        runningCostModel.addCost("ctrlRegGrav", uRegCost, 5e-1)
        runningCostModel.addCost("shoulderOrientation", imuArmOrientationCost, 5e1)
        runningCostModel.addCost("wristOrientation", frameOrientationCost, 5e1)
        
    
        # Create Differential Action Model (DAM), i.e. continuous dynamics and cost functions
        running_DAM = crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuation, runningCostModel)
        runningModel.append(crocoddyl.IntegratedActionModelEuler(running_DAM, dt))

    terminal_DAM = crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuation, runningCostModel)
    
    # Create Integrated Action Model (IAM), i.e. Euler integration of continuous dynamics and cost
    
    terminalModel = crocoddyl.IntegratedActionModelEuler(terminal_DAM, 0.)
        
    # Create the shooting problem
    problem = crocoddyl.ShootingProblem(x0, runningModel, terminalModel)
    
    # Create solver + callbacks
    # ddp = SolverSQP(problem)
    ddp = crocoddyl.SolverDDP(problem)
    ddp.setCallbacks([crocoddyl.CallbackLogger()])
    ddp.use_filter_line_search = True
    
    # Warm start : initial state + gravity compensation
    xs_init = [x0 for i in range(T+1)]
    us_init = ddp.problem.quasiStatic(xs_init[:-1])
    
    # Solve
    ddp.solve(xs_init, us_init, maxiter=10)
    
    return ddp